<a href="https://colab.research.google.com/github/yadavrishikesh/BayesNF/blob/main/EcoCounter_all_predictions_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary packages
!pip install -q bayesnf cartopy contextily geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00


In [2]:

import warnings
warnings.simplefilter('ignore')

import contextily as ctx
import geopandas as gpd
import jax
import numpy as np
import pandas as pd
import os
import datetime
from urllib.parse import urlparse
from bayesnf.spatiotemporal import BayesianNeuralFieldMAP


In [3]:

def run_model(train_url, test_url):
    """
    Trains a Bayesian Neural Field MAP model on specified train data and tests on test data.

    Parameters:
    - train_url (str): URL to the training dataset CSV file.
    - test_url (str): URL to the test dataset CSV file.

    Returns:
    - predictions_df (DataFrame): DataFrame containing true values, predicted values, and quantiles.
    """
    # Determine base directory from train_url and create unique outputs directory
    train_name = os.path.basename(urlparse(train_url).path).split('.')[0]
    test_name = os.path.basename(urlparse(test_url).path).split('.')[0]
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(os.getcwd(), 'outputs', f'{train_name}_{test_name}_{timestamp}')
    os.makedirs(output_dir, exist_ok=True)

    # Download and load datasets
    !wget -q {train_url} -O train_data.csv
    !wget -q {test_url} -O test_data.csv
    df_train = pd.read_csv('train_data.csv', index_col=0, parse_dates=['datetime'])
    df_test = pd.read_csv('test_data.csv', index_col=0, parse_dates=['datetime'])

    # Log-transform response in training data
    df_train_log = df_train.copy()
    df_train_log['response'] = np.log1p(df_train_log['response'])

    # Define and train the Bayesian Neural Field model
    model = BayesianNeuralFieldMAP(
        width=512,
        depth=2,
        freq='D',
        seasonality_periods=['W'],
        num_seasonal_harmonics=[2],
        feature_cols=['datetime', 'lat', 'lon', 'elev', 'walkscore', 'num_ppo', 'temp', 'precp', 'visbl', 'wsp'],
        target_col='response',
        observation_model='NORMAL',
        timetype='index'
    )

    model = model.fit(df_train_log,
                      seed=jax.random.PRNGKey(0),
                      ensemble_size=32,
                      num_epochs=2500)

    # Process test data
    df_test_cleaned = df_test.dropna(subset=['response'])
    df_test_cleaned_log = df_test_cleaned.copy()
    df_test_cleaned_log['response'] = np.log1p(df_test_cleaned_log['response'])

    # Make predictions
    yhat, yhat_quantiles = model.predict(df_test_cleaned, quantiles=(0.025, 0.5, 0.975))

    # Compile results into DataFrame
    predictions_df = pd.DataFrame({
        'datetime': df_test_cleaned.index,
        'True_Values': df_test_cleaned_log['response'],
        'Predicted_Mean': yhat_quantiles[1],
        'Quantile_0.025': yhat_quantiles[0],
        'Quantile_0.975': yhat_quantiles[2]
    })

    # Save output in the unique outputs directory
    output_path = os.path.join(output_dir, 'predictions_output.csv')
    predictions_df.to_csv(output_path, index=False)

    return predictions_df




In [4]:
# spatial interpolation
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Eco_pred-type_spatIntpl.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Eco_pred-type_spatIntpl.csv'
predictions_df = run_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,2874.0,6.545656,5.237208,9.154754
2,2,1600.0,6.020605,4.840631,8.464034
3,3,1176.0,5.726278,4.462554,8.539927
4,4,2285.0,6.556521,5.195445,8.665833
5,5,322.0,3.918273,2.214289,6.557671


In [5]:
# forecasting
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Eco_pred-type_forecast.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Eco_pred-type_forecast.csv'

predictions_df = run_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,693.0,8.139323,7.199327,8.979779
2,2,762.0,7.205187,6.682689,7.876945
3,3,455.0,6.213091,5.452715,6.905539
4,4,190.0,5.445563,4.532623,6.146601
5,5,165.0,6.854400,6.015817,7.655406


In [6]:
# space-time prediction
train_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/train_data_Eco_pred-type_spaceTime.csv'
test_data_url = 'https://raw.githubusercontent.com/yadavrishikesh/BayesNF/main/data/test_data_Eco_pred-type_spaceTime.csv'

predictions_df = run_model(train_data_url, test_data_url)
predictions_df.head()

,datetime,True_Values,Predicted_Mean,Quantile_0.025,Quantile_0.975
1,1,3240,7.958107,6.344917,9.992222
2,2,3646,7.155617,5.157404,9.423456
3,3,2171,6.228010,4.370378,8.329934
4,4,1303,5.504339,3.217742,7.885087
5,5,1146,6.860358,5.100767,9.187068


In [7]:
# Mount Google Drive to access it from Colab
from google.colab import drive
drive.mount('/content/drive')

# Copy the 'outputs' directory to Google Drive
!cp -r /content/outputs /content/drive/MyDrive/BayesNF/outputs_Eco

Mounted at /content/drive
